# Components with hierarchy


![](https://i.imgur.com/3pczkyM.png)


You can define some components (waveguides, bends, couplers) as a stand alone components, with basic input parameters (width, length, radius ...)

Then you can re-use those components in a more complex hiearchical components.

`gdsfactory` does this by passing the higher level component with the lower level functions to build the components.

You can customize any of the functions thanks to `functools.partial`

In [ ]:
import gdsfactory as gf

# gf.CONF.plotter = 'holoviews'

In [ ]:
@gf.cell
def bend_with_straight(
    bend=gf.components.bend_euler,
    straight=gf.components.straight,
) -> gf.Component:
    c = gf.Component()
    b = bend()
    s = straight()

    bref = c << b
    sref = c << s

    sref.connect("o2", bref.ports["o2"])
    c.info["length"] = b.info["length"] + s.info["length"]
    return c


c = bend_with_straight()
print(c.metadata.info.length)
c.plot()

Lets **customize** the functions that we pass.
For example, we want to increase the radius of the bend from the default 10um to 20um.

In [ ]:
c = gf.components.bend_circular()
c.plot()

## functools.partial

Partial lets you define different default parameters for a function, so you can modify the settings for the child cells.

In [ ]:
from functools import partial

In [ ]:
bend20 = partial(gf.components.bend_circular, radius=20)
b = bend20()
b.plot()

In [ ]:
type(bend20)

In [ ]:
bend20.func.__name__

In [ ]:
bend20.keywords

In [ ]:
b = bend_with_straight(bend=bend20)
print(b.metadata.info.length)
b.plot()

In [ ]:
# You can still modify the bend to have any bend radius
b3 = bend20(radius=10)
b3.plot()

## PDK custom fab

You can define a new PDK by creating function that customize partial parameters of the generic functions.

Lets say that this PDK uses layer (41, 0) for the pads (instead of the layer used in the generic pad function).

You can also access `functools.partial` from `gf.partial`

In [ ]:
import gdsfactory as gf

pad = gf.partial(gf.components.pad, layer=(41, 0))

In [ ]:
c = pad()
c.plot()

## Composing functions

You can combine more complex functions out of smaller functions.

Lets say that we want to add tapers and grating couplers to a wide waveguide.

In [ ]:
c1 = gf.components.straight()
c1.plot()

In [ ]:
straight_wide = gf.partial(gf.components.straight, width=3)
c3 = straight_wide()
c3.plot()

In [ ]:
c1 = gf.components.straight(width=3)
c1.plot()

In [ ]:
c2 = gf.add_tapers(c1)
c2.plot()

In [ ]:
c2.metadata_child.changed  # You can still access the child metadata

In [ ]:
c3 = gf.routing.add_fiber_array(c2, with_loopback=False)
c3.plot()

In [ ]:
c3.metadata_child.changed  # You can still access the child metadata

Lets do it with a **single** step thanks to `toolz.pipe`

In [ ]:
import toolz

add_fiber_array = gf.partial(gf.routing.add_fiber_array, with_loopback=False)
add_tapers = gf.add_tapers

# pipe is more readable than the equivalent add_fiber_array(add_tapers(c1))
c3 = toolz.pipe(c1, add_tapers, add_fiber_array)
c3.plot()

we can even combine `add_tapers` and `add_fiber_array` thanks to `toolz.compose` or `toolz.compose`

For example:

In [ ]:
add_tapers_fiber_array = toolz.compose_left(add_tapers, add_fiber_array)
c4 = add_tapers_fiber_array(c1)
c4.plot()

is equivalent to

In [ ]:
c5 = add_fiber_array(add_tapers(c1))
c5.plot()

as well as equivalent to

In [ ]:
add_tapers_fiber_array = toolz.compose(add_fiber_array, add_tapers)
c6 = add_tapers_fiber_array(c1)
c6.plot()

or

In [ ]:
c7 = toolz.pipe(c1, add_tapers, add_fiber_array)
c7.plot()

In [ ]:
c7.metadata_child.changed  # You can still access the child metadata

In [ ]:
c7.metadata.child.child.name, c7.metadata.child.child.function_name

In [ ]:
c7.metadata.child.name, c7.metadata.child.function_name

In [ ]:
c7.metadata.name, c7.metadata.function_name

In [ ]:
c7.metadata.changed.keys()